In [53]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import cross_validation
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [54]:
tr = pd.read_csv('input/train_v2.csv')
#tr.info()

In [55]:
te = pd.read_csv('input/sampleSubmission_v2.csv')
#te.info()

In [56]:
#te[:5]

In [57]:
te.sponsored = np.nan

In [58]:
#te.info()

In [59]:
t = pd.concat([tr, te])
#t.info()

In [60]:
pr1 = pd.read_csv('tmp/prec_rec_1.csv')
#pr1.info()

In [61]:
#pr1.describe().T

In [62]:
x = pd.merge(t, pr1, on='file', how='left', suffixes=('','_1'))
#x.info()

In [63]:
pr2 = pd.read_csv('tmp/prec_rec_2.csv')
#pr2.info()

In [64]:
#pr2.describe().T

In [65]:
y = pd.merge(x, pr2, on='file', how='left', suffixes=('','_2'))
#y.info()

In [66]:
pr3 = pd.read_csv('tmp/prec_rec_3.csv')
#pr3.info()

In [67]:
#pr3.describe().T

In [68]:
z = pd.merge(y, pr3, on='file', how='left', suffixes=('','_3'))
#z.info()

In [69]:
c = pd.read_csv('tmp/count_features.csv')
#c.info()

In [70]:
#c.describe().T

In [71]:
w = pd.merge(z, c, on='file', how='left')
#w.info()

In [72]:
w.index = w.file
del w['file']

In [73]:
for c in filter(lambda c: c.startswith('n_links_'), w.columns.tolist()):
    w[c + '_prop'] = 1.0 * w[c] / w.n_links
for c in filter(lambda c: c.startswith('n_imgs_'), w.columns.tolist()):
    w[c + '_prop'] = 1.0 * w[c] / w.n_imgs

In [84]:
for i in range(len(w.columns)):
    print i, w.columns[i]

0 sponsored
1 p100
2 r100
3 p300
4 r300
5 p1000
6 r1000
7 p3000
8 r3000
9 p10000
10 r10000
11 p30000
12 r30000
13 p100_2
14 r100_2
15 p300_2
16 r300_2
17 p1000_2
18 r1000_2
19 p3000_2
20 r3000_2
21 p10000_2
22 r10000_2
23 p30000_2
24 r30000_2
25 p100_3
26 r100_3
27 p300_3
28 r300_3
29 p1000_3
30 r1000_3
31 p3000_3
32 r3000_3
33 p10000_3
34 r10000_3
35 p30000_3
36 r30000_3
37 n_title_chars
38 n_title_words
39 n_links
40 n_links_rel
41 n_links_abs
42 n_links_domain_com
43 n_links_domain_org
44 n_links_scheme_none
45 n_links_scheme_http
46 n_links_scheme_https
47 n_links_path_nonempty
48 n_imgs
49 n_imgs_rel
50 n_imgs_abs
51 n_imgs_fmt_jpg
52 n_imgs_fmt_gif
53 n_imgs_fmt_png
54 n_paragraphs
55 n_chars
56 n_words
57 avg_word_length
58 n_lines
59 n_spaces
60 n_tabs
61 n_braces
62 n_brackets
63 n_dashes
64 n_dots
65 n_bangs
66 n_eqs
67 n_pluses
68 n_x_pars
69 n_links_rel_prop
70 n_links_abs_prop
71 n_links_domain_com_prop
72 n_links_domain_org_prop
73 n_links_scheme_none_prop
74 n_links_sche

In [74]:
#w.describe().T

In [75]:
tr = w[pd.notnull(w.sponsored)]
tr = tr.fillna(0)
#tr.info()

In [76]:
trl = tr.sponsored
del tr['sponsored']

In [77]:
te = w[pd.isnull(w.sponsored)]
del te['sponsored']
te = te.fillna(0)
#te.info()

In [78]:
m = RandomForestClassifier(n_estimators=500, n_jobs=-1)
#m = KNeighborsClassifier(n_neighbors=5)
#m = SVC(probability=True)

In [79]:
scores = cross_validation.cross_val_score(m, tr, trl, cv=3, scoring='roc_auc')

In [80]:
# RandomForestClassifier(n_estimators=500, max_depth=30, oob_score=True, n_jobs=-1):
#     array([ 0.90357458,  0.90298073,  0.90189454])
# KNeighborsClassifier(n_neighbors=5):
#     array([ 0.67784316,  0.68113898,  0.6737584 ])
scores

array([ 0.91198505,  0.9123692 ,  0.9111538 ])

In [81]:
#m.fit(tr, trl)

In [82]:
# preds = DataFrame(m.predict_proba(te))[1]
# sub = DataFrame({'file': te.index, 'sponsored': preds})
# sub.info()

In [83]:
# sub.to_csv('output/prec_rec_count_randfor_02.csv', index=False)